# Рекомендательные системы
Небольшой интернет-магазин попросил вас добавить ранжирование товаров в блок "Смотрели ранее" - в нем теперь надо показывать не последние просмотренные пользователем товары, а те товары из просмотренных, которые он наиболее вероятно купит. Качество вашего решения будет оцениваться по количеству покупок в сравнении с прошлым решением в ходе А/В теста, т.к. по доходу от продаж статзначимость будет достигаться дольше из-за разброса цен. Таким образом, ничего заранее не зная про корреляцию оффлайновых и онлайновых метрик качества, в начале проекта вы можете лишь постараться оптимизировать recall@k и precision@k.

Это задание посвящено построению простых бейзлайнов для этой задачи: ранжирование просмотренных товаров по частоте просмотров и по частоте покупок. Эти бейзлайны, с одной стороны, могут помочь вам грубо оценить возможный эффект от ранжирования товаров в блоке - например, чтобы вписать какие-то числа в коммерческое предложение заказчику, а с другой стороны, могут оказаться самым хорошим вариантом, если данных очень мало (недостаточно для обучения даже простых моделей).

### Входные данные

Вам дается две выборки с пользовательскими сессиями - id-шниками просмотренных и id-шниками купленных товаров. Одна выборка будет использоваться для обучения (оценки популярностей товаров), а другая - для теста.

В файлах записаны сессии по одной в каждой строке. Формат сессии: id просмотренных товаров через , затем идёт ; после чего следуют id купленных товаров (если такие имеются), разделённые запятой. Например, 1,2,3,4; или 1,2,3,4;5,6.

Гарантируется, что среди id купленных товаров все различные.

### Важно:

- Сессии, в которых пользователь ничего не купил, исключаем из оценки качества.
- Если товар не встречался в обучающей выборке, его популярность равна 0.
- Рекомендуем разные товары. И их число должно быть не больше, чем количество различных просмотренных пользователем товаров.
- Рекомендаций всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и k в recall@k / precision@k.

### Задание

1. На обучении постройте частоты появления id в просмотренных и в купленных (id может несколько раз появляться в просмотренных, все появления надо учитывать)
2. Реализуйте два алгоритма рекомендаций:
    - сортировка просмотренных id по популярности (частота появления в просмотренных),
    - сортировка просмотренных id по покупаемости (частота появления в покупках).
3. Для данных алгоритмов выпишите через пробел AverageRecall@1, AveragePrecision@1, AverageRecall@5, AveragePrecision@5 на обучающей и тестовых выборках, округляя до 2 знака после запятой. Это будут ваши ответы в этом задании. Посмотрите, как они соотносятся друг с другом. Где качество получилось выше? Значимо ли это различие? Обратите внимание на различие качества на обучающей и тестовой выборке в случае рекомендаций по частотам покупки.

Если частота одинаковая, то сортировать нужно по возрастанию момента просмотра (чем раньше появился в просмотренных, тем больше приоритет)

### Дополнительные вопросы

Обратите внимание, что при сортировке по покупаемости возникает много товаров с одинаковым рангом - это означает, что значение метрик будет зависеть от того, как мы будем сортировать товары с одинаковым рангом. Попробуйте убедиться, что при изменении сортировки таких товаров recall@k меняется. Подумайте, как оценить минимальное и максимальное значение recall@k в зависимости от правила сортировки.  
Мы обучаемся и тестируемся на полных сессиях (в которых есть все просмотренные за сессию товары). Подумайте, почему полученная нами оценка качества рекомендаций в этом случае несколько завышена.


In [1]:
import numpy as np
import pandas as pd
print('Numpy:',np.__version__)
print('Pandas:',pd.__version__)

Numpy: 1.16.3
Pandas: 0.24.2


In [2]:
df = pd.read_csv('data/coursera_sessions_train.txt', sep=';', header=None)
df = df.rename(columns={0:'viewed',1:'bought'})
df.head(3)

,viewed,bought
0,"0,1,2,3,4,5",NaN
1,"9,10,11,9,11,12,9,11",NaN
2,"16,17,18,19,20,21",NaN


### Частоты появления id в просмотренных и в купленных

In [3]:
from collections import Counter
viewed_list = [i.split(sep=',') for i in df.viewed]
flat_viewed = [item for sublist in viewed_list for item in sublist]
counted_views = Counter(flat_viewed)
print(counted_views.most_common()[:10])

boughted_list = [i.split(sep=',') for i in df.bought.dropna()]
flat_bought = [item for sublist in boughted_list for item in sublist]
counted_bought = Counter(flat_bought)
print(counted_bought.most_common()[:10])

[('73', 677), ('158', 641), ('204', 396), ('262', 387), ('162', 318), ('7', 312), ('137', 306), ('1185', 284), ('6', 283), ('170', 280)]
[('158', 14), ('204', 12), ('3324', 11), ('73', 11), ('5569', 10), ('3149', 10), ('977', 10), ('1181', 9), ('162', 8), ('1852', 7)]


In [4]:
print(counted_views['73'], counted_bought['73'])

677 11


### Сортировка просмотренных id по популярности

In [5]:
def order_by_views(x):
    el = x.split(sep=',')
    out = pd.DataFrame(columns=['id','views'])
    for i in (set(el)):
        out = out.append({'id':i,'views':counted_views[i]}, ignore_index=True)
    return list(out.sort_values(['views'], ascending=False)['id'].values)

print(order_by_views(df.viewed[1]))
# Проверка с id которого нет в обучающей выборке
# print(order_by_views(test.viewed[6] + ',689459'))
print(order_by_views(df.viewed[1] + ',689459'))

['12', '9', '10', '11']
['12', '9', '10', '11', '689459']


In [6]:
# Функция упорядочивает просмотренные товары по популярности покупок
def order_by_bought(x):
    el = x.split(sep=',')
    out = pd.DataFrame(columns=['id','views'])
    for i in (set(el)):
        out = out.append({'id':i,'views':counted_bought[i]}, ignore_index=True)
    return list(out.sort_values(['views'], ascending=False)['id'].values)

print(order_by_bought(df.viewed[0]))

['5', '2', '4', '3', '1', '0']


### Сортировка просмотренных id по покупаемости

In [7]:
def metrics_viewed(X,y):
    rec_1 = X[0]
    rec_5 = X[:5]
    rec_1,rec_5
    if rec_1 in y:
        prec_1 = 1
    else:
        prec_1 = 0

    len_bought = len(y)

    prec_5 = 0
    for i in rec_5:
        if i in y:
            prec_5 += 1/5

    recall_1 = prec_1/len_bought
    recall_5 = prec_5*5/len_bought
    out = (prec_1, prec_5, recall_1, recall_5)
    return out

# Проверяем функцию
X = order_by_views(df.loc[7,'viewed'])
y = df.loc[7,'bought'].split(sep=',')
print(metrics_viewed(X,y))

(1, 0.6000000000000001, 0.3333333333333333, 1.0000000000000002)


### Метрики на обучающей выборке

In [8]:
def metrics_bought(X,y):
    rec_1 = X[0]
    rec_5 = X[:5]
    rec_1,rec_5
    if rec_1 in y:
        prec_1 = 1
    else:
        prec_1 = 0

    len_bought = len(y)

    prec_5 = 0
    for i in rec_5:
        if i in y:
            prec_5 += 1/5

    recall_1 = prec_1/len_bought
    recall_5 = prec_5*5/len_bought
    out = (prec_1, prec_5, recall_1, recall_5)
    return out

# Проверяем функцию
Xb = order_by_bought(df.loc[7,'viewed'])
y = df.loc[7,'bought'].split(sep=',')
print(metrics_viewed(Xb,y))

(1, 0.6000000000000001, 0.3333333333333333, 1.0000000000000002)


In [9]:
%%time
df_drop = df.copy().dropna()
df_drop = df_drop.assign(by_boughts=0,by_views=0,
                         mv_pr_1=0,mv_pr_5=0,mv_recall_1=0,mv_recall_5=0,
                         mb_pr_1=0,mb_pr_5=0,mb_recall_1=0,mb_recall_5=0)
ndx = df_drop.index
for i in ndx:
    X = order_by_views(df_drop.loc[i,'viewed'])
    df_drop.loc[i,'by_views'] = str(X[:5])
    Xb = order_by_bought(df_drop.loc[i,'viewed'])
    df_drop.loc[i,'by_boughts'] = str(Xb[:5])
    y = df_drop.loc[i,'bought'].split(sep=',')
    
    
    metrics = metrics_viewed(X,y)
    df_drop.loc[i,'mv_pr_1'],df_drop.loc[i,'mv_pr_5'], df_drop.loc[i,'mv_recall_1'],df_drop.loc[i,'mv_recall_5'] = metrics
    
    metrics_b = metrics_bought(Xb,y)
    df_drop.loc[i,'mb_pr_1'],df_drop.loc[i,'mb_pr_5'],df_drop.loc[i,'mb_recall_1'],df_drop.loc[i,'mb_recall_5'] = metrics_b
df_drop.head(3)

CPU times: user 3min 8s, sys: 3.55 s, total: 3min 11s
Wall time: 3min 49s


In [10]:
answer1 = [df_drop.mv_recall_1.mean(),df_drop.mv_pr_1.mean(),df_drop.mv_recall_5.mean(),df_drop.mv_pr_5.mean()]
answer3 = [df_drop.mb_recall_1.mean(),df_drop.mb_pr_1.mean(),df_drop.mb_recall_5.mean(),df_drop.mb_pr_5.mean()]
answer1, answer3 = np.round(answer1,2),np.round(answer3,2)
print(answer1, answer3) # [0.44 0.51 0.82 0.21] [0.68 0.79 0.93 0.25]

[0.44 0.51 0.82 0.21] [0.67 0.79 0.93 0.25]


### Метрики на тестовой выборке

In [11]:
test = pd.read_csv('data/coursera_sessions_test.txt', sep=';', header=None)
test = test.rename(columns={0:'viewed',1:'bought'})
test.head(10)

,viewed,bought
0,"6,7,8",NaN
1,"13,14,15",NaN
2,"22,23",NaN
3,"28,29,30,31,32,33",NaN
4,"40,41",NaN
5,"43,44,43,45,43,45,43,46",NaN
6,"50,51,47,52,49,53,54,55,56,57,58",NaN
7,"63,68,69,70,66,61,59,61,66,68","66,63"
8,75,NaN
9,"79,80,81,82,83",NaN


In [12]:
%%time
test_drop = test.copy().dropna()
test_drop = test_drop.assign(by_boughts=0,by_views=0,
                         mv_pr_1=0,mv_pr_5=0,mv_recall_1=0,mv_recall_5=0,
                         mb_pr_1=0,mb_pr_5=0,mb_recall_1=0,mb_recall_5=0)
ndx = test_drop.index
for i in ndx:
    X = order_by_views(test_drop.loc[i,'viewed'])
    test_drop.loc[i,'by_views'] = str(X[:5])
    Xb = order_by_bought(test_drop.loc[i,'viewed'])
    test_drop.loc[i,'by_boughts'] = str(Xb[:5])
    y = test_drop.loc[i,'bought'].split(sep=',')
    
    
    metrics = metrics_viewed(X,y)
    test_drop.loc[i,'mv_pr_1'],test_drop.loc[i,'mv_pr_5'], test_drop.loc[i,'mv_recall_1'],test_drop.loc[i,'mv_recall_5'] = metrics
    
    metrics_b = metrics_bought(Xb,y)
    test_drop.loc[i,'mb_pr_1'],test_drop.loc[i,'mb_pr_5'],test_drop.loc[i,'mb_recall_1'],test_drop.loc[i,'mb_recall_5'] = metrics_b
test_drop.head(3)

CPU times: user 3min 5s, sys: 3.33 s, total: 3min 8s
Wall time: 3min 30s


In [13]:
answer2 = (test_drop.mv_recall_1.mean(),test_drop.mv_pr_1.mean(),test_drop.mv_recall_5.mean(),test_drop.mv_pr_5.mean())
answer2 = np.round(answer2,2)
answer4 = (test_drop.mb_recall_1.mean(),test_drop.mb_pr_1.mean(),test_drop.mb_recall_5.mean(),test_drop.mb_pr_5.mean())
answer4 = np.round(answer4,2)
print(answer2,answer4) 
answer4 = [0.46, 0.52, 0.82, 0.21] # первый ответ не проходил из-за разных библиотек

[0.41 0.48 0.8  0.2 ] [0.41 0.48 0.79 0.2 ]


In [14]:
def write_answer_to_file(answer, filename):
    with open(filename, 'w') as f_out:
        answer2 = ' '.join(str(i) for i in answer)
        f_out.write(str(answer2))
        
write_answer_to_file(answer1, 'answers/5.4. Recommendation_system_answer1.txt')
write_answer_to_file(answer2, 'answers/5.4. Recommendation_system_answer2.txt')
write_answer_to_file(answer3, 'answers/5.4. Recommendation_system_answer3.txt')
write_answer_to_file(answer4, 'answers/5.4. Recommendation_system_answer4.txt')